In [8]:
import os
from nltk.parse import stanford
import nltk

In [10]:
# Dependency Tree
from nltk.parse.stanford import StanfordDependencyParser
dep_parser=StanfordDependencyParser(model_path="/home/shanu/nltk/jars/englishPCFG.ser.gz")

In [11]:
def lca(tree, index1, index2):
    node = index1
    path1 = []
    path2 = []
    path1.append(index1)
    path2.append(index2)
    while(node != tree.root):
        node = tree.nodes[node['head']]
        path1.append(node)
    node = index2
    while(node != tree.root):
        node = tree.nodes[node['head']]
        path2.append(node)
    for l1, l2 in zip(path1[::-1],path2[::-1]):
        if(l1==l2):
            temp = l1
    return temp

In [12]:
def path_lca(tree, node, lca_node):
    path = []
    path.append(node)
    while(node != lca_node):
        node = tree.nodes[node['head']]
        path.append(node)
    return path

In [13]:
import _pickle 
f = open('../data/training_data', 'rb')
sentences, e1, e2 = _pickle.load(f)
f.close()

In [ ]:
sentences[7588] = 'The reaction mixture is kept in the dark at room temperature for 1.5 hours .'
sentences[2608] = "This strawberry sauce has about a million uses , is freezer-friendly , and is so much better than that jar of Smuckers strawberry sauce that you 've had sitting in your fridge since that time you made banana splits 1.5 years ago ."

In [ ]:
## Uncomment this for test set. 
# sentences[2590] = "The pendant with the bail measure 1.25'' ."
# sentences[2664] = "The cabinet encloses a 6.5 inch cone woofer , 4 inch cone midrange , and a 0.86 inch balanced dome tweeter ."

In [20]:
word_path1 = []
word_path2 = []
rel_path1 = []
rel_path2 = []
pos_path1 = []
pos_path2 = []
for i in range(8000):
    word_path1.append(0)
    word_path2.append(0)
    rel_path1.append(0)
    rel_path2.append(0)
    pos_path1.append(0)
    pos_path2.append(0)

In [38]:
for i in range(8000):
    try:
        parse_tree = dep_parser.raw_parse(sentences[i])
        for trees in parse_tree:
            tree = trees
        node1 = tree.nodes[e1[i]+1]
        node2 = tree.nodes[e2[i]+1]
        if node1['address']!=None and node2['address']!=None:
            print(i, "success")
            lca_node = lca(tree, node1, node2)
            path1 = path_lca(tree, node1, lca_node)
            path2 = path_lca(tree, node2, lca_node)

            word_path1[i] = [p["word"] for p in path1]
            word_path2[i] = [p["word"] for p in path2]
            rel_path1[i] = [p["rel"] for p in path1]
            rel_path2[i] = [p["rel"] for p in path2]
            pos_path1[i] = [p["tag"] for p in path1]
            pos_path2[i] = [p["tag"] for p in path2]
        else:
            print(i, node1["address"], node2["address"])
    except AssertionError:
        print(i, "error")
    

7588 success


In [39]:
file = open('../data/train_paths', 'wb')
_pickle.dump([word_path1, word_path2, rel_path1, rel_path2, pos_path1, pos_path2], file)